导入必要的 `numpy` 库

In [ ]:
import numpy as np

定义抽象基类 `Layer`，机器相关的接口

In [ ]:
class Layer:
    def __init__(self):
        pass
    
    def forward(self, input):
        pass

    def backward(self, output_gradient, learning_rate):
        pass

定义类`Dense`, Dense 类中包含两个成员：
- *weights*: 权重， 权重是一个二维向量， 其形状是 (output_size, input_size)
- *biases*: 偏置， 偏置是一个一维向量， 其形状是 (output_size)

Dense 类中主要的方法是
- *forward*: 用于计算前向推到
- *backward*: 用于做反向传播，调整权重。


In [ ]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.biases

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        self.biases -= learning_rate * output_gradient
        self.weights -= learning_rate * weights_gradient
        return np.dot(self.weights.T, output_gradient)

    def __repr__(self):
        return f'weights {self.weights}\nbiases {self.biases}\n'

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

In [ ]:
def predict(network, input):
    output = input
    for layer in network:
        #print(f'output.shape {output.shape}, layer {layer}')
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 5000, learning_rate = 0.01, interval=100, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)
            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        if verbose and (e + 1) % interval == 0:
            print(f"{e + 1}/{epochs}, error={error}")
    
    print(f"{epochs}/{epochs}, error={error}")

In [ ]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)
        
    def __repr__(self):
        return 'Activation: Tanh\n'

In [ ]:
network = [
    Dense(7,16),
    Tanh(),
    Dense(16,10),
]

In [ ]:
print(network)

In [ ]:
x0 = [1, 1, 1,0 ,1 ,1, 1] #0
x1 = [0, 0, 1,0 ,0 ,1, 0] #1
x2 = [1, 0, 1,1 ,1 ,0, 1] #2
x3 = [1, 0, 1,1 ,0 ,1, 1] #3
x4 = [0, 1, 1,1 ,0 ,1, 0] #4
x5 = [1, 1, 0,1 ,0 ,1, 1] #5
x6 = [1, 1, 0,1 ,1 ,1, 1] #6
x7 = [1, 0, 1,0 ,0 ,1, 0] #7
x8 = [1, 1, 1,1 ,1 ,1, 1] #8
x9 = [1, 1, 1,1 ,0 ,1, 1] #9
x_train = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9]
y_train = [[1 if i == j else 0 for i in range(10)] for j in range(10)]

x_train = np.array(x_train).reshape(10, 7, 1)
y_train = np.array(y_train).reshape(10, 10, 1)

In [ ]:
x_input = np.array(x4).reshape(7,1)
output = predict(network, x_input)
print(f'Before training, output is: {output}, predict x is :{ np.argmax(output)}')

In [ ]:
train(network, mse, mse_prime, x_train, y_train, epochs = 1000, learning_rate = 0.01, interval=10, verbose = True)

In [ ]:
x_input = np.array(x4).reshape(7,1)
output = predict(network, x_input)
print(f'output is: {output}, predict x is :{ np.argmax(output)}')

In [ ]:
print(network)